# Highlights and steps taken in the LLM Response Filtering project:-

1) The below is model uses Facebook's blenderbot to generate responses to user
text. I started off with this and later realized that it can only give proper responses for questions and conversations.

2) So I though of adding a model which will factually check the authenticity of the response. But then I hit another problem where the model could give a confident response even for some things that it knows very little about.

3) So I decided to rate the model's response through a confidence threshold and whatever response which has a confidence score below the threshold will be discarded and then replace with "I don't know".

4) The reason to work on this particular problem is because I feel like the halucination caused by these models will create a lot of damage to infromation in the society in future. As more people start to use these chatbots, the top 1% wealthy might take advantage of the misinformation and then can move things in the way they wanted.

5) By implementing these methods we can be able to atleast say that an infroamtion that the chatbot provides is true or not. That is the reason why I later used Wikipedia to enable our conversational model to generate factual responses.

6) **The project is under development...... I am now currently figuring out how to add context window to the chatbot within the program to be able to remember what the user asked and what it told the user.**

In [5]:
# Install required libraries in Colab
!pip install transformers torch wikipedia --quiet

import torch
import wikipedia
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Load Fact-Based QA Model (Replaces BERT NLI)
print("Loading fact-checking model...")
fact_checker = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Load Chatbot Model (Facebook BlenderBot)
print("Loading chatbot model...")
chatbot_model_name = "facebook/blenderbot-400M-distill"  # Lightweight model for chat
chatbot_tokenizer = AutoTokenizer.from_pretrained(chatbot_model_name)
chatbot_model = AutoModelForSeq2SeqLM.from_pretrained(chatbot_model_name)

# Confidence Thresholds
SOFTMAX_THRESHOLD = 0.30  # Softmax probability confidence

# Bad Words Filter
BAD_WORDS = {"fuck", "shit", "bitch", "asshole", "dumbass", "bimbo", "stupid", "idiot"}

def detect_bad_words(text):
    """Check for bad words in user input."""
    words = text.lower().split()
    return any(word in BAD_WORDS for word in words)

def softmax_confidence(logits):
    """Computes softmax probabilities and returns the highest confidence score."""
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
    return torch.max(probs).item()  # Get the highest probability

def get_factual_answer(question):
    """Fetches answers from Wikipedia if the chatbot lacks confidence."""
    try:
        return wikipedia.summary(question, sentences=1)
    except:
        return None  # No Wikipedia answer found

def generate_response(prompt):
    """Generate chatbot response and filter based on confidence."""
    inputs = chatbot_tokenizer(prompt, return_tensors="pt")
    output = chatbot_model.generate(**inputs, max_length=100, return_dict_in_generate=True, output_scores=True)

    generated_text = chatbot_tokenizer.decode(output.sequences[0], skip_special_tokens=True)
    confidence = softmax_confidence(output.scores[0])  # Compute softmax confidence

    if confidence < SOFTMAX_THRESHOLD:
        # Try getting a factual answer before returning "I don't know"
        factual_answer = get_factual_answer(prompt)
        if factual_answer:
            return factual_answer

        return "I don’t know about this, can you please provide more context?"

    return generated_text

print("\nChatbot Ready! Type 'exit' to quit.")

while True:
    user_input = input("\nYou: ").strip()

    if user_input.lower() == "exit":
        print("Goodbye!")
        break

    if detect_bad_words(user_input):
        print("Bot: Don't talk bad words to me, bimbo!!!!")
        continue

    bot_response = generate_response(user_input)
    print(f"Bot: {bot_response}")


Loading fact-checking model...


Device set to use cpu


Loading chatbot model...

Chatbot Ready! Type 'exit' to quit.


KeyboardInterrupt: Interrupted by user